In [1]:
import pandas as pd
import numpy as np
import pandas_ta as ta
import yfinance as yf

import pandas_ta as ta

from bokeh.plotting import figure, show
from bokeh.io import output_notebook, curdoc

from bokeh.models import NumeralTickFormatter, DatetimeTickFormatter
from bokeh.models import Span, HoverTool

In [2]:
output_notebook()

Loading BokehJS ...

# Obtendo a cotação

In [3]:
df = yf.download('CSAN3.SA', period='1y')
df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-08-19,22.000000,22.750000,21.799999,22.440001,21.128448,9891100
2021-08-20,22.139999,22.700001,21.900000,22.530001,21.213188,6953100
2021-08-23,22.600000,22.610001,22.100000,22.120001,20.827152,5809900
2021-08-24,22.709999,23.290001,22.400000,22.480000,21.166109,10317100
2021-08-25,22.700001,22.700001,22.120001,22.590000,21.269682,10677500
...,...,...,...,...,...,...
2022-08-15,20.440001,21.330000,20.440001,21.080000,21.080000,5744300
2022-08-16,21.129999,21.320000,20.620001,21.000000,21.000000,5477700
2022-08-17,20.889999,21.440001,20.840000,21.270000,21.270000,6489300


# Tratando os dados e gerando alguns indicadores

In [4]:
data = df.copy()

# Alvo da operação
target = 2

data.drop('Close', axis=1, inplace=True)
data.rename(columns={'Adj Close': 'Close'}, inplace=True)

data['rsi'] = data.ta.rsi(6)
data['ema'] = data.ta.ema(9)
data['week_day'] = data.index.day_of_week
data['target'] = data['close'].shift(-target)

# Criando filtros e calculando a diferença percentual

In [5]:
filter1 = (data['rsi'] > 50) # RSI(6) acima de 50
filter2 = (data['close'] > data['ema']) # Fechamento acima da EMA(9)
filter3 = ((data['week_day'] == 1)) # Abrir operação na terça-feira

# Cálculo da **Diferença Percentual**
value1 = (data['target'] - data['close'])
value2 = ((data['target'] + data['close']) / 2 )
percent_diff = (value1/value2)*100

data.loc[filter1 & filter2 & filter3, 'result'] = round(percent_diff, 2)

# Gerando as estatísticas

In [6]:
total = data['result'].dropna().shape[0]
acertos = data['result'].dropna().loc[data['result'] > 0]
erros = data['result'].dropna().loc[data['result'] < 0]

inicio_backtest = data.index[0].strftime('%d/%m/%Y')
fim_backtest = data.index[-1].strftime('%d/%m/%Y')

percent_acerto = round((acertos.shape[0]/total)*100, 2)
percent_erro = round((erros.shape[0]/total)*100, 2)

acc = data['result'].fillna(0).cumsum().iloc[-1]
max_drawdown = round(ta.max_drawdown(data['result'].fillna(0).cumsum()), 2)

media_acertos = round(acertos.mean(), 2)

media_errors = round(erros.mean(), 2)

payoff = round(media_acertos/abs(media_errors), 2)

# Exibindo gráfico e resultado final

In [7]:
TOOLS = 'pan,wheel_zoom,crosshair,box_zoom,reset,save'
TOOLTIPS = [
    ("Data", "@x{%F}"),
    ("Acc", "@y{0.2f}%"),
]

FORMATTER = {
    '@x'  : 'datetime', # use 'datetime' formatter for 'date' field
}

hover = HoverTool(
    tooltips=TOOLTIPS,
    formatters=FORMATTER,
    # display a tooltip whenever the cursor is vertically in line with a glyph
    mode='vline'
)     

curdoc().theme = "caliber"

# create a new plot with a title and axis labels
p = figure(title="Acumulado do Setup - CSAN3",
           x_axis_label='Data',
           y_axis_label='Acumulado %',
           tools=TOOLS,
           plot_width=800, plot_height=400)

p.title.align = "center"
p.title.text_font_size = "20px"
p.xaxis[0].formatter = DatetimeTickFormatter(days="%d/%m/%Y", )

p.line(data.index, data['result'].fillna(0).cumsum(), line_width=1, alpha=0.7, color='black')
zero_value = Span(location=0,
                    dimension='width', line_color='#FF9E73',
                    line_dash='dashed', line_width=2.5, line_alpha=1.)

p.add_layout(zero_value)
p.add_tools(hover)


# show the results
show(p)

print('\n ***** RESULTADO FINAL *****')
print(f'\nPeríodo do Backtest: {inicio_backtest} a {fim_backtest}')

print(f'\nTotal de trades: {total}')
print(f'Acertos: {acertos.shape[0]}')
print(f'Erros: {erros.shape[0]}')

print(f'\nPercentual de acertos: {percent_acerto}%')
print(f'Percentual de erros: {percent_erro}%')

print(f'\nMédia de trades com gain: {media_acertos}%')
print(f'Média de trades com loss: {media_errors}%')

print(f'\nPayoff: {payoff}')

print(f'\nAcumulado final: {acc}%')
print(f'Máximo Drawdown: {max_drawdown}%')


 ***** RESULTADO FINAL *****

Período do Backtest: 19/08/2021 a 19/08/2022

Total de trades: 21
Acertos: 18
Erros: 3

Percentual de acertos: 85.71%
Percentual de erros: 14.29%

Média de trades com gain: 3.05%
Média de trades com loss: -2.05%

Payoff: 1.49

Acumulado final: 48.69%
Máximo Drawdown: 5.89%
